In [68]:
import pandas as pd

In [69]:
ACCESS_KEY='<ACCESS_KEY>'
SECRET_KEY='<SECRET_KEY>'
BUCKET_NAME='<BUCKET_NAME>'

# Store Raw Data

## Read Data

In [70]:
sales=pd.read_csv('Archive/SalesData.csv')
costs=pd.read_csv('Archive/costs.csv')
countries=pd.read_csv('Archive/Country_Table.csv')
customers=pd.read_csv('Archive/Customer_Table.csv')
fx_table=pd.read_csv('Archive/FX_Table.csv')

## Store Data

In [71]:
import pandas as pd
import boto3

def store_data(df, file_key, bucket_name, aws_access_key, aws_secret_key):
    df.to_csv(f"s3://{bucket_name}/{file_key}", index=False, storage_options={"key": aws_access_key, "secret": aws_secret_key})    

store_data(sales, "SalesData.csv", BUCKET_NAME, ACCESS_KEY, SECRET_KEY)
store_data(costs, "costs.csv", BUCKET_NAME, ACCESS_KEY, SECRET_KEY)
store_data(countries, "Country_Table.csv", BUCKET_NAME, ACCESS_KEY, SECRET_KEY)
store_data(customers, "Customer_Table.csv", BUCKET_NAME, ACCESS_KEY, SECRET_KEY)
store_data(fx_table, "FX_Table.csv", BUCKET_NAME, ACCESS_KEY, SECRET_KEY)

ValueError: Bucket create failed '<BUCKET_NAME>': Parameter validation failed:
Invalid bucket name "<BUCKET_NAME>": Bucket name must match the regex "^[a-zA-Z0-9.\-_]{1,255}$" or be an ARN matching the regex "^arn:(aws).*:(s3|s3-object-lambda):[a-z\-0-9]*:[0-9]{12}:accesspoint[/:][a-zA-Z0-9\-.]{1,63}$|^arn:(aws).*:s3-outposts:[a-z\-0-9]+:[0-9]{12}:outpost[/:][a-zA-Z0-9\-]{1,63}[/:]accesspoint[/:][a-zA-Z0-9\-]{1,63}$"

# Raw Data ETL

## Extract

In [72]:
def read_data(file_key, bucket_name, aws_access_key, aws_secret_key):
    return pd.read_csv(f"s3://{bucket_name}/{file_key}", storage_options={"key": aws_access_key, "secret": aws_secret_key})

sales = read_data("SalesData.csv", BUCKET_NAME, ACCESS_KEY, SECRET_KEY)
costs = read_data("costs.csv", BUCKET_NAME, ACCESS_KEY, SECRET_KEY)
countries = read_data("Country_Table.csv", BUCKET_NAME, ACCESS_KEY, SECRET_KEY)
customers = read_data("Customer_Table.csv", BUCKET_NAME, ACCESS_KEY, SECRET_KEY)
fx_table = read_data("FX_Table.csv", BUCKET_NAME, ACCESS_KEY, SECRET_KEY)

ParamValidationError: Parameter validation failed:
Invalid bucket name "<BUCKET_NAME>": Bucket name must match the regex "^[a-zA-Z0-9.\-_]{1,255}$" or be an ARN matching the regex "^arn:(aws).*:(s3|s3-object-lambda):[a-z\-0-9]*:[0-9]{12}:accesspoint[/:][a-zA-Z0-9\-.]{1,63}$|^arn:(aws).*:s3-outposts:[a-z\-0-9]+:[0-9]{12}:outpost[/:][a-zA-Z0-9\-]{1,63}[/:]accesspoint[/:][a-zA-Z0-9\-]{1,63}$"

## Transform

#### date lookup table

In [73]:
# create date range
date_lookup = pd.DataFrame({"date": pd.date_range('01/01/2000', '31/12/2050')})

# first_day_of_month
date_lookup['first_day_of_month']=date_lookup['date'].apply(lambda x : x.replace(day=1))

# first_day_of_week
date_lookup['first_day_of_week'] = date_lookup['date'].dt.to_period('W').apply(lambda r: r.start_time)

# date_key
date_lookup['date_key'] = date_lookup['date'].dt.strftime("%d/%m/%Y").apply(hash)

date_lookup.head()

,date,first_day_of_month,first_day_of_week,date_key
0,2000-01-01,2000-01-01,1999-12-27,-7873389636989488394
1,2000-01-02,2000-01-01,1999-12-27,8883797052972681791
2,2000-01-03,2000-01-01,2000-01-03,7404711964508303880
3,2000-01-04,2000-01-01,2000-01-03,-1378614078894463865
4,2000-01-05,2000-01-01,2000-01-03,7323148345004735751


#### Sales Data

In [74]:
sales.head()

,category,subcategory,product_family,key_product,sku,description,grade,country_id,cost_currency,cost_per_device,sales_date,sold_currency,price_sold_per_device,status,customer_id,quantity,sales_order_id,serial,bin_id
0,Computers,Apple,Mac Pro (Late 2019),Mac Pro (Late 2019) 2TB SSD,CP-AP-R071RA-2TS-W3245-A,Mac Pro Xeon W 3.2GHz Rack (Late 2019) 2TB SSD,A,5,USD,7351,18/06/2022,USD,8454,Sold,100198,53,2201685,268471,UE3477
1,Smartphones,Apple,iPhone 12,iPhone 12 128GB,PH-AP-IP1200-128-B,iPhone 12 128GB,B,20,GBP,335,23/06/2022,GBP,599,Sold,100168,27,2219375,952379,CK9815
2,Laptops,Apple,"MacBook Air 7,2 13 Inch (2015/17)","MacBook Air 7,2 13 Inch (2015/17) 256GB SSD",LP-AP-A07213-25S-5350U-B,MacBook Air i5 1.8GHz 13 inch (2017) 256GB SSD,B,11,USD,219,26/06/2022,USD,458,Sold,100181,24,2220215,727694,WZ6443
3,Smartphones,Apple,iPhone 11,iPhone 11 64GB,PH-AP-IP1100-064-C,iPhone 11 64GB,C,77,EUR,229,22/06/2022,EUR,300,Sold,100300,68,2206360,474162,CQ7097
4,Smartphones,Apple,iPhone 11,iPhone 11 128GB,PH-AP-IP1100-128-Z,iPhone 11 128GB,Z,77,EUR,307,23/06/2022,EUR,360,Sold,100116,71,2202925,589342,SK7391


In [75]:
# create a surrogate keys --> hash is not the best function for hashing
sales['product_id'] = sales['sku'].apply(hash)
sales['status_id'] = sales['status'].apply(hash)
sales['cost_currency_id'] = sales['cost_currency'].apply(hash)
sales['sold_currency_id'] = sales['sold_currency'].apply(hash)
sales['date_key'] = sales['sales_date'].apply(hash)

In [76]:
# create product dimension table
product_cols = ['category', 'subcategory', 'product_family', 'key_product', 'sku', 'description', 'grade']
product = sales[['product_id'] + product_cols].drop_duplicates('product_id', keep=False) # drop duplicates based on product_id
product.head()

,product_id,category,subcategory,product_family,key_product,sku,description,grade
0,-1439248536363157107,Computers,Apple,Mac Pro (Late 2019),Mac Pro (Late 2019) 2TB SSD,CP-AP-R071RA-2TS-W3245-A,Mac Pro Xeon W 3.2GHz Rack (Late 2019) 2TB SSD,A
106,5512411016230074324,Smartphones,Apple,iPhone 12,iPhone 12 64GB,PH-AP-IP1200-064-D,iPhone 12 64GB,D
423,5000164413682653830,Laptops,Apple,Macbook Pro (Mid 2017) 15 inch,Macbook Pro (Mid 2017) 15 inch 1TB SSD,LP-AP-P14315-01S-7920H-C,MacBook Pro i7 3.1GHz 15 inch (Mid 2017) 1TB SSD,C
736,4941242922755433850,Laptops,Apple,"MacBook Air 6,2 13 Inch (Early 2014)","MacBook Air 6,2 13 Inch (Early 2014) 256GB SSD",LP-AP-A06213-25S-4250U-C,MacBook Air Core i5 1.3GHz 13 inch (Mid 2013) ...,C
744,-5135914494770584065,Tablets,Apple,"iPad Pro 12.9"" 4th Generation","iPad Pro 12.9"" 4th Generation 512GB",TB-AP-PADP4T-FAC-512-D,iPad Pro 12.9 Inch 4th Gen 512GB WiFi & Cellular,D


In [77]:
# create currency dimension table
currency_cols = ['sold_currency']
currency = sales[['sold_currency_id'] + currency_cols].drop_duplicates('sold_currency_id').rename(columns={"sold_currency_id": "id"}) # drop duplicates based on sold_currency_id
currency.head()

,id,sold_currency
0,-1074992787815396821,USD
1,4985948176599916422,GBP
3,-571006599092188678,EUR
19,3112220886281541028,JPY
21,-2319899775838855190,AUD


In [78]:
# create status dimension table
status_cols = ['status']
status = sales[['status_id'] + status_cols].drop_duplicates('status_id').rename(columns={'status_id': 'id'}) # drop duplicates based on status_id
status.head()

,id,status
0,-3773042236790807944,Sold
19,-8261799544735215941,Not finalised


In [79]:
# create sales fact table
sales = sales.drop(product_cols+ currency_cols + status_cols + ['cost_currency', 'sales_date'], axis=1)
sales.head()

,country_id,cost_per_device,price_sold_per_device,customer_id,quantity,sales_order_id,serial,bin_id,product_id,status_id,cost_currency_id,sold_currency_id,date_key
0,5,7351,8454,100198,53,2201685,268471,UE3477,-1439248536363157107,-3773042236790807944,-1074992787815396821,-1074992787815396821,4398136639123285220
1,20,335,599,100168,27,2219375,952379,CK9815,-3915168727598716723,-3773042236790807944,4985948176599916422,4985948176599916422,6983656433815983893
2,11,219,458,100181,24,2220215,727694,WZ6443,-4606809883592676889,-3773042236790807944,-1074992787815396821,-1074992787815396821,-5054677733452352981
3,77,229,300,100300,68,2206360,474162,CQ7097,-3730219937214677271,-3773042236790807944,-571006599092188678,-571006599092188678,-3377042288321706278
4,77,307,360,100116,71,2202925,589342,SK7391,5386026829982668081,-3773042236790807944,-571006599092188678,-571006599092188678,6983656433815983893


#### costs data

In [80]:
costs.head()

,Instance type,db.r5.4xlarge($),db.r6g.xlarge($),db.r6g.4xlarge($),db.r6g.large($),db.r5.xlarge($),No instance type($),db.r5.large($),db.t3.medium($),db.t4g.medium($),db.t2.medium($),Total costs($)
0,Instance type total,11243.52,9515.062437,8985.588373,8759.031866,6568.915021,5182.736341,2524.337511,1998.32908,1538.272428,408.456,56724.249057
1,2023-10-01,1904.64,1033.985662,627.638541,1198.302979,1904.640000,925.364037,476.160000,392.83200,253.510053,69.192,8786.265272
2,2023-11-01,1843.20,1650.240000,1649.520000,1425.610932,1390.588622,890.067754,460.800000,380.16000,168.480000,66.960,9925.627308
3,2023-12-01,1904.64,1705.248000,1704.504000,1514.827441,1051.152355,903.665290,476.160000,392.83200,174.096000,69.192,9896.317086
4,2024-01-01,1904.64,1705.248000,1704.504000,1576.624653,952.320000,894.093694,476.160000,392.83200,174.096000,69.192,9849.710347


In [81]:
# clean the costs dataset
rename_dict={
    'db.r5.4xlarge($)': 'r5.4xlarge',
    'db.r6g.xlarge($)': 'r6g.xlarge',
    'db.r6g.4xlarge($)': 'r6g.4xlarge',
    'db.r6g.large($)': 'r6g.large',
    'db.r5.xlarge($)': 'r5.xlarge',
    'No instance type($)': 'No instance type',
    'db.r5.large($)': 'r5.large',
    'db.t3.medium($)': 't3.medium',
    'db.t4g.medium($)': 't4g.medium',
    'db.t2.medium($)': 't2.medium',
    'Instance type': 'date'
}

# drop the second row, total cost column, and rename the columns
costs = costs.drop(index=costs.index[0], axis=0).drop('Total costs($)', axis=1).rename(columns=rename_dict)

In [82]:
# reshape the costs dataset from wide to long
costs = costs.melt(id_vars=['date'], 
           var_name='instance_type', 
           value_name='cost($)')

In [83]:
costs.head()

,date,instance_type,cost($)
0,2023-10-01,r5.4xlarge,1904.64
1,2023-11-01,r5.4xlarge,1843.20
2,2023-12-01,r5.4xlarge,1904.64
3,2024-01-01,r5.4xlarge,1904.64
4,2024-02-01,r5.4xlarge,1781.76


#### fx_table

In [84]:
fx_table.head()

,Date,GBPUSD,EURUSD,AUDUSD,EURGBP,EURAUD,GBPAUD,JPYUSD,JPYAUD,JPYEUR,...,HKDGBP,HKDJPY,HKDKRW,TWDUSD,TWDAUD,TWDEUR,TWDGBP,TWDJPY,TWDKRW,TWDHKD
0,06/06/2022,1.253,1.069,0.719,0.854,1.487,1.742,0.0076,0.0105,0.0071,...,0.102,16.818,159.932,0.0340,0.0472,0.0318,0.0271,4.482,42.620,0.266
1,30/05/2022,1.265,1.078,0.719,0.852,1.498,1.759,0.0078,0.0109,0.0073,...,0.101,16.259,157.568,0.0345,0.0480,0.0320,0.0273,4.402,42.654,0.271
2,16/05/2022,1.232,1.044,0.698,0.847,1.496,1.767,0.0078,0.0111,0.0074,...,0.103,16.436,163.129,0.0336,0.0482,0.0322,0.0273,4.341,43.086,0.264
3,02/05/2022,1.250,1.051,0.705,0.841,1.491,1.772,0.0077,0.0109,0.0073,...,0.102,16.585,161.538,0.0339,0.0481,0.0323,0.0272,4.417,43.018,0.266
4,18/04/2022,1.301,1.078,0.736,0.829,1.465,1.768,0.0079,0.0107,0.0073,...,0.098,16.202,157.446,0.0342,0.0465,0.0317,0.0263,4.347,42.245,0.268


In [85]:
# reshape the fx_table from wide to long
fx_table = fx_table.melt(id_vars=['Date'], 
           var_name='conversion', 
           value_name='conversion_values')

In [86]:
fx_table.head()

,Date,conversion,conversion_values
0,06/06/2022,GBPUSD,1.253
1,30/05/2022,GBPUSD,1.265
2,16/05/2022,GBPUSD,1.232
3,02/05/2022,GBPUSD,1.250
4,18/04/2022,GBPUSD,1.301


## Load

Now we have 9 dataframes map to 9 tables in our target database. We can load them using df.to_sql()

In [87]:
# loaded to countries table
countries.head() 

,id,city,country,sales_channel
0,5,Chicago,US,B2B
1,20,London,GB,B2C
2,11,Chicago,US,B2C
3,77,Dublin,IE,B2B
4,16,London,GB,B2B


In [88]:
# loaded to customers table
customers.head()

,id,customer_name
0,100198,Rempel - Will
1,100168,Rohan - Kulas
2,100181,"Kertzmann, Hagenes and Runolfsson"
3,100300,"Bruen, Gaylord and Moore"
4,100116,"Hilll, Beer and Upton"


In [89]:
# loaded to date_lookup table
date_lookup.head()

,date,first_day_of_month,first_day_of_week,date_key
0,2000-01-01,2000-01-01,1999-12-27,-7873389636989488394
1,2000-01-02,2000-01-01,1999-12-27,8883797052972681791
2,2000-01-03,2000-01-01,2000-01-03,7404711964508303880
3,2000-01-04,2000-01-01,2000-01-03,-1378614078894463865
4,2000-01-05,2000-01-01,2000-01-03,7323148345004735751


In [90]:
# loaded to product table
product.head()

,product_id,category,subcategory,product_family,key_product,sku,description,grade
0,-1439248536363157107,Computers,Apple,Mac Pro (Late 2019),Mac Pro (Late 2019) 2TB SSD,CP-AP-R071RA-2TS-W3245-A,Mac Pro Xeon W 3.2GHz Rack (Late 2019) 2TB SSD,A
106,5512411016230074324,Smartphones,Apple,iPhone 12,iPhone 12 64GB,PH-AP-IP1200-064-D,iPhone 12 64GB,D
423,5000164413682653830,Laptops,Apple,Macbook Pro (Mid 2017) 15 inch,Macbook Pro (Mid 2017) 15 inch 1TB SSD,LP-AP-P14315-01S-7920H-C,MacBook Pro i7 3.1GHz 15 inch (Mid 2017) 1TB SSD,C
736,4941242922755433850,Laptops,Apple,"MacBook Air 6,2 13 Inch (Early 2014)","MacBook Air 6,2 13 Inch (Early 2014) 256GB SSD",LP-AP-A06213-25S-4250U-C,MacBook Air Core i5 1.3GHz 13 inch (Mid 2013) ...,C
744,-5135914494770584065,Tablets,Apple,"iPad Pro 12.9"" 4th Generation","iPad Pro 12.9"" 4th Generation 512GB",TB-AP-PADP4T-FAC-512-D,iPad Pro 12.9 Inch 4th Gen 512GB WiFi & Cellular,D


In [91]:
# loaded to currency table
currency.head()

,id,sold_currency
0,-1074992787815396821,USD
1,4985948176599916422,GBP
3,-571006599092188678,EUR
19,3112220886281541028,JPY
21,-2319899775838855190,AUD


In [92]:
# loaded to status table
status.head()

,id,status
0,-3773042236790807944,Sold
19,-8261799544735215941,Not finalised


In [93]:
# loaded to sales table
sales.head()

,country_id,cost_per_device,price_sold_per_device,customer_id,quantity,sales_order_id,serial,bin_id,product_id,status_id,cost_currency_id,sold_currency_id,date_key
0,5,7351,8454,100198,53,2201685,268471,UE3477,-1439248536363157107,-3773042236790807944,-1074992787815396821,-1074992787815396821,4398136639123285220
1,20,335,599,100168,27,2219375,952379,CK9815,-3915168727598716723,-3773042236790807944,4985948176599916422,4985948176599916422,6983656433815983893
2,11,219,458,100181,24,2220215,727694,WZ6443,-4606809883592676889,-3773042236790807944,-1074992787815396821,-1074992787815396821,-5054677733452352981
3,77,229,300,100300,68,2206360,474162,CQ7097,-3730219937214677271,-3773042236790807944,-571006599092188678,-571006599092188678,-3377042288321706278
4,77,307,360,100116,71,2202925,589342,SK7391,5386026829982668081,-3773042236790807944,-571006599092188678,-571006599092188678,6983656433815983893


In [94]:
# loaded to costs table
costs.head()

,date,instance_type,cost($)
0,2023-10-01,r5.4xlarge,1904.64
1,2023-11-01,r5.4xlarge,1843.20
2,2023-12-01,r5.4xlarge,1904.64
3,2024-01-01,r5.4xlarge,1904.64
4,2024-02-01,r5.4xlarge,1781.76


In [95]:
# loaded to currency_converter table
fx_table.head()

,Date,conversion,conversion_values
0,06/06/2022,GBPUSD,1.253
1,30/05/2022,GBPUSD,1.265
2,16/05/2022,GBPUSD,1.232
3,02/05/2022,GBPUSD,1.250
4,18/04/2022,GBPUSD,1.301
